In [5]:
import os
import zipfile
import pandas as pd
import re

In [7]:
import os
import zipfile
import pandas as pd

def search_and_save_files(folder_path):
    # Initialize an empty list to collect results
    results = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.zip'):
                zip_file_path = os.path.join(root, file)
                
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_file_list = zip_ref.namelist()
                    
                    for zip_file_name in zip_file_list:
                        with zip_ref.open(zip_file_name) as zipped_file:
                            content = None
                            for encoding in ['utf-8', 'utf-16', 'latin-1']:
                                try:
                                    content = zipped_file.read().decode(encoding)
                                    break
                                except UnicodeDecodeError:
                                    continue
                            
                            if content is None:
                                continue
                            
                            # Check if the first 1000 characters contain 'amendment' or 'amended'
                            first_1000_chars = content[:1000].lower()
                            if any(keyword in first_1000_chars for keyword in ['amendment', 'amended', 'amends', 'restatement', 'restated', 'restates']):
                                continue

                            search_terms = ['credit agreement', 'loan agreement', 'credit facility',
                                            'loan and security agreement', 'loan & security agreement',
                                            'revolving credit', 'financing and security agreement',
                                            'financing & security agreement', 'credit and guarantee agreement',
                                            'credit & guarantee agreement', 'credit and security agreement', 'credit & security agreement']
                            toc_terms = ['table of contents', 'tableofcontents', 'table of contents', 'tableofcontents']
                            toc_within_lines = 5000

                            # Search for terms and check for 'table of contents' nearby
                            for term in search_terms:
                                term_lower = term.lower()
                                if term_lower in content.lower():
                                    index = content.lower().find(term_lower)
                                    for toc_term in toc_terms:
                                        if content.lower().find(toc_term, index, index + toc_within_lines) != -1:
                                            # Append result to the list
                                            results.append({'File Name': zip_file_name, 'Search Term': term})
                                            break
                                    break
    
    # Create DataFrame from results list
    df = pd.DataFrame(results)
    return df

folder_path = '/Users/zrsong/Dropbox (MIT)/Research Projects/Direct Lending/credit_agreements/Original_loan_contracts'
result_df = search_and_save_files(folder_path)

In [8]:
# save result_df to a csv file
result_df.to_csv('../Outputs/Agreements.csv', index=False)